# TCP Congestion Control et Bufferbloat

In this exercise, you will create your own network simulation to study TCP dynamics and how seemingly minor configuration decisions made by network operators can have major effects on performance.

TCP is a protocol that enables reliable transmission over an unreliable packet-switched network. Another important component of TCP is congestion control, which limits the sending rates of the end host to prevent the network infrastructure from being overwhelmed by traffic.

However, networks can suffer from performance issues related to congestion, even when end hosts are using TCP. One such issue, known as bufferbloat, can occur when packet buffers on routers and switches are too large.

In this lab, you will use Mininet, a useful tool for network experiments, to emulate a small network and collect various performance statistics relevant to TCP congestion control and bufferbloat. This will allow you to reason about the effects of TCP and router configuration on network performance. Additionally, it will introduce you to the powerful tool that is Mininet.

## Background

#### TCP Congestion Window Size
The TCP congestion window size parameter, commonly referred to as "cwnd," is managed by the sender and determines the amount of traffic that can be in-flight (sent but unacknowledged) at any given time. There are numerous algorithms for controlling the value of cwnd during a TCP connection, all aimed at maximizing throughput while avoiding congestion. The additive increase and multiplicative decrease algorithm was discussed in class.

#### Bufferbloat
Bufferbloat is a phenomenon that occurs when a switching device is configured to use excessively large buffers, which in turn can cause high latency and packet delay variation (jitter). This can happen even in a typical home network like this:


<img width=600 src="figures/home-network.png">

Here, the end host of the home network is connected to the home router. The home router is then connected, via cable or DSL, to a head-end router managed by the Internet Service Provider (ISP). By simulating and experimenting with a similar network in Mininet, you will see how bufferbloat leads to poor performance.

You can read more about the bufferbloat issue and how it almost took the Internet down here: 
* https://www.wired.com/2012/05/van-jacobson/ 
* https://queue.acm.org/detail.cfm?id=2209336

#### Mininet

While we have been using Mininet for a couple classes already, here is a "new" introduction to it :)

Mininet is a network emulator that allows you to create a custom network of virtual hosts, switches, controllers, and links, all on a single computer. The virtual devices in the emulated network can run real programs; anything that can run on Linux can also run on a Mininet device. This makes Mininet a valuable tool for quickly and easily simulating network protocols and measurements. This [Introduction to Mininet](https://github.com/mininet/mininet/wiki/Introduction-to-Mininet) is a useful guide to get started with Mininet’s Python API. The [Mininet website](http://www.mininet.org) has additional resources if you are interested.

## Part A: Network Simulation and Measurement
To begin, you must first create the following network using Mininet’s Python API, which emulates a typical home network:

<img width=450 src="figures/mininet-topo.png">

Here, h1 is a web server that has a fast connection (1 Gb/s) to your home router. The home router has a slow downstream connection (1.5 Mb/s) to your home computer. The round-trip propagation delay, or the minimum RTT between h1 and h2, is 20 ms. The router’s buffer memory size (queue) will be the independent variable set in your simulation.

To create a custom topology in Mininet, we extend the `mininet.topo.Topo` class. We have already added the switch (the router) to the topology for you. You need to add h1, h2, and links with the appropriate characteristics to create the setup specified in the image above. The first subsections of ["Working with Mininet"](https://github.com/mininet/mininet/wiki/Introduction-to-Mininet#working) describe how to add elements to a topology and define performance parameters.

In [23]:
from mininet.topo import Topo

class BBTopo(Topo):
    "Simple topology for bufferbloat experiment."

    def __init__(self, queue_size):
        super(BBTopo, self).__init__()
        
        # Create switch s0 (the router)
        self.addSwitch('s0')
        
        # TODO: Create two hosts with names 'h1' and 'h2'

        
        # TODO: Add links with appropriate bandwidth, delay, and queue size parameters. 
        #       Set the router queue size using the queue_size argument
        #       Set bandwidths/latencies using the bandwidths and minimum RTT given in the network diagram above
        
        
        return

Next, we need a couple of helper functions to generate traffic between the two hosts. The following function starts a long-lived TCP flow which sends data from h1 to h2 using **iperf**. [Iperf](https://iperf.fr/) is "a tool for active measurements of the maximum achievable bandwidth on IP networks."  You can think of this iperf traffic like a one-way video call. It continually attempts to send a high volume of traffic from the web server h1 to the home computer h2. 

The following function receives one argument called `net`, which is an instance of mininet with a BBTopo topology that we have created above. We have written the part for the iperf server (h2). Notice that in iperf, server is the one that receives data, which should be the home computer h2. You need to complete the function to also start iperf on the iperf client (h1). The iperf session should run for the number of seconds given in the `experiment_time` argument.

You will need to use the `popen` function to run shell commands on a mininet host. The first argument to `popen` is a string command just like you would run in your shell. The second argument should be `shell=True`. You will need to look up the appropriate command line options to run iperf as a client for a given amount of time in the documentation here: [https://iperf.fr/iperf-doc.php#3doc](https://iperf.fr/iperf-doc.php#3doc). You will also need to include the IP address of h2 in your iperf command. This IP address can be accessed with the `h2.IP()` method.  

In [24]:
def start_iperf(net, experiment_time):
    # Start a TCP server on host 'h2' using perf. 
    # The -s parameter specifies server mode
    # The -w 16m parameter ensures that the TCP flow is not receiver window limited (not necessary for client)
    print "Starting iperf server"
    h2 = net.get('h2')
    server = h2.popen("iperf -s -w 16m", shell=True)
    
    # TODO: Start an TCP client on host 'h1' using iperf. 
    #       Ensure that the client runs for experiment_time seconds
    print "Starting iperf client"


Next, you need to complete the following function, which starts a consecutive ping train from h1 to h2 to measure the RTTs. A ping should be sent every 0.1 seconds. The results should be redirected from stdout to the `outfile` argument.

As before, `net` is an instance of Mininet with a BBTopo topology. As previously, you will need to use `popen`. The command argument of `popen` can redirect stdout using `>` like a normal shell command. Refer to the `ping` manual page for more details on the available command-line arguments. Make sure the second argument of `popen` is `shell=True`.

In [25]:
def start_ping(net, outfile="pings.txt"):
    # TODO: Start a ping train from h1 to h2 with 0.1 seconds between pings, redirecting stdout to outfile
    print "Starting ping train"


Next, we develop helper functions to measure the congestion window of TCP traffic. This will allow us to analyze the dynamics of TCP connections in the Mininet network. The following functions are already completed.

In [26]:
from subprocess import Popen
import os

def start_tcpprobe(outfile="cwnd.txt"):
    Popen("sudo cat /proc/net/tcpprobe > " + outfile, shell=True)

def stop_tcpprobe():
    Popen("killall -9 cat", shell=True).wait()

We then create a helper function that monitors the queue length on a given interface. This will allow us to analyze how the number of packets in the router's buffer queues affects performance. This function is already completed.

In [27]:
from multiprocessing import Process
from monitor import monitor_qlen

def start_qmon(iface, interval_sec=0.1, outfile="q.txt"):
    monitor = Process(target=monitor_qlen,
                      args=(iface, interval_sec, outfile))
    monitor.start()
    return monitor

Nous avons également besoin d'une fonction helper qui démarre un serveur Web sur h1. Cette fonction est déjà terminée.

In [28]:
from time import sleep

def start_webserver(net):
    h1 = net.get('h1')
    proc = h1.popen("python http/webserver.py", shell=True)
    sleep(1)
    return [proc]

Finally, we need a helper function that runs on h2, retrieves the website from h1 every 3 seconds for `experiment_time`, and prints the average and standard deviation of the download times. This function is already completed.

In [29]:
from time import time
from numpy import mean, std
from time import sleep

def fetch_webserver(net, experiment_time):
    h2 = net.get('h2')
    h1 = net.get('h1')
    download_times = []
    
    start_time = time()
    while True:
        sleep(3)
        now = time()
        if now - start_time > experiment_time:
            break
        fetch = h2.popen("curl -o /dev/null -s -w %{time_total} ", h1.IP(), shell=True)
        download_time, _ = fetch.communicate()
        print "Download time: {0}, {1:.1f}s left...".format(download_time, experiment_time - (now-start_time))
        download_times.append(float(download_time))
        
    average_time = mean(download_times)
    std_time = std(download_times)
    print "\nDownload Times: {}s average, {}s stddev\n".format(average_time, std_time)

Now, we need to put all the pieces together to create the network, start all the traffic, and perform the measurements.

The following `bufferbloat()` function should:
* create a `BBTopo` object
* start the TCP and queue monitors
* start a long-running TCP flow using iperf
* start the ping train
* start the web server
* periodically download the index.html web page from h1 and measure how long it takes to retrieve it

Note that the long-running flow, the ping train, and the web server downloads must all happen simultaneously. Once you’ve completed the assignment steps up to this point, complete the sections marked `TODO` in the `bufferbloat()` function below. Each TODO section requires adding a line to call a function defined above.

In [30]:
from mininet.node import CPULimitedHost, OVSController
from mininet.link import TCLink
from mininet.net import Mininet
from mininet.log import lg, info
from mininet.util import dumpNodeConnections

from time import time
import os
from subprocess import call

def bufferbloat(queue_size, experiment_time, experiment_name):
    # Don't forget to use the arguments!
    
    # Set the cwnd control algorithm to "reno" (half cwnd on 3 duplicate acks)
    #    Modern Linux uses CUBIC-TCP by default that doesn't have the usual sawtooth
    #    behaviour.  For those who are curious, replace reno with cubic
    #    see what happens...
    os.system("sysctl -w net.ipv4.tcp_congestion_control=reno")
    
    # create the topology and network
    topo = BBTopo(queue_size)
    net = Mininet(topo=topo, host=CPULimitedHost, link=TCLink, 
                  controller= OVSController)
    net.start()

    # Print the network topology 
    dumpNodeConnections(net.hosts)
    
    # Performs a basic all pairs ping test to ensure the network set up properly
    net.pingAll()
    
    # Start monitoring TCP cwnd size
    outfile = "{}_cwnd.txt".format(experiment_name)
    start_tcpprobe(outfile)

    # TODO: Start monitoring the queue sizes with the start_qmon() function.
    #       Fill in the iface argument with "s0-eth2" if the link from s0 to h2
    #       is added second in BBTopo or "s0-eth1" if the link from s0 to h2
    #       is added first in BBTopo. This is because we want to measure the 
    #       number of packets in the outgoing queue from s0 to h2. 
    outfile = "{}_qsize.txt".format(experiment_name)
    qmon = start_qmon(iface="TODO", outfile=outfile)

    # TODO: Start the long lived TCP connections with the start_iperf() function

    
    # TODO: Start pings with the start_ping() function
    outfile = "{}_pings.txt".format(experiment_name)

    
    # TODO: Start the webserver with the start_webserver() function

    
    # TODO: Measure and print website download times with the fetch_webserver() function

    
    # Stop probing 
    stop_tcpprobe()
    qmon.terminate()
    net.stop()
    
    # Ensure that all processes you create within Mininet are killed.
    Popen("pgrep -f webserver.py | xargs kill -9", shell=True).wait()
    call(["mn", "-c"])

Once you have completed all the steps above, use the `bufferbloat()` function to run the experiment twice: once with a queue size of 20 packets, and then with a queue size of 100 packets. Make sure to run the experiments long enough to observe TCP dynamics, such as the sawtooth behavior of cwnd, in your results (300 seconds should be sufficient). Choose `experiment_name` arguments that reflect the queue size.

In [31]:
from subprocess import call
call(["mn", "-c"])

# TODO: call the bufferbloat function twice, once with queue size of 20 packets and once with a queue size of 100.



0

## Part B: Plotting the Results

In this section, you will analyze your measurements by plotting the variations of the congestion window, queue length, and ping RTT over time. We have provided plotting functions for each of these measurements, which are called in the already completed `plot_measurements()` function below.

In [32]:
%matplotlib inline
from plot_cwnd import plot_congestion_window
from plot_qsize import plot_queue_length
from plot_ping import plot_ping_rtt

def plot_measurements(experiment_name_list, cwnd_histogram=False):
    
    # plot the congestion window over time
    for name in experiment_name_list:
        cwnd_file = "{}_cwnd.txt".format(name)
        plot_congestion_window(cwnd_file, histogram=cwnd_histogram)
    
    # plot the queue size over time
    for name in experiment_name_list:
        qsize_file = "{}_qsize.txt".format(name)
        plot_queue_length(qsize_file)
    
    # plot the ping RTT over time
    for name in experiment_name_list:
        ping_file = "{}_pings.txt".format(name)
        plot_ping_rtt(ping_file)

You now need to call the `plot_measurements` function so that the `experiment_name_list` argument is the list of `experiment_name` arguments you used to run `bufferbloat()` above. This should generate 6 plots with the results of the experiments.

In [33]:
#TODO: Call plot_measurements() to plot your results


## Part C: Analysis

In this part of the assignment, you will answer a few questions about TCP and bufferbloat using your simulations and the plots from the previous section. These questions are intentionally open-ended, and many have multiple correct answers. There is no required response length, but try to be both thorough and concise. 1-2 sentences is probably too short, while more than 2-3 paragraphs is likely too long.

First, take some time to reflect on the simulation you just performed. The simulation was set up as a home network with a home computer connected to a remote server via a router. The link from the router to the server had a much lower bandwidth than the link from the home computer to the router. The independent variable in the simulation was the maximum length of the packet buffer waiting to be sent from the router to the server.

There were three sources of traffic:
1. A long-running TCP session (created using iperf) sending a high volume of traffic from the home computer to the server.
2. Regularly spaced pings and ping responses to and from the home computer and the server.
3. Regularly spaced attempts to download a webpage (using HTTP over TCP) from the personal computer to the server.

As you (hopefully) discovered during the experiment, increasing the packet buffer length on the router significantly reduced the performance of the ping RTT and HTTP download rate metrics.

### Questions

#### Q1.
What other computer networks besides a home network might have a configuration similar to the one you simulated?

#### A1.
*TODO: give your answer here.*

#### Q2.
Write a symbolic equation to describe the relationship between RTT and queue size.

The symbolic equation should be generalized to any queue size. Essentially, consider a snapshot of a system at a given moment and use the queue size and link delays parametrically to calculate the RTT.

An example of a (incorrect) symbolic equation: 
$$RTT = kq^2$$
where $k$ is a constant factor and $q$ is the number of packets in the queue. Your equation is not limited to $k$ and $q$.

#### A2.
*TODO: your answer here. Use single dollar signs for inline LaTeX math formatting and double dollar signs for block LaTeX math formatting.*

#### Q3.  
Describe in technical terms why increasing the buffer size reduces performance (RTT and webpage download times), leading to the bufferbloat effect. Be sure to explicitly reference the plots you generated and the relationship between TCP congestion control and buffer size.

#### A3.
*TODO: your answer here.*

#### Q4.  
Rephrase the cause of the bufferbloat effect using a non-technical analogy unrelated to computer networking. It is important to be able to describe technical content in a way that a layperson can understand, and generating analogies often helps your own reasoning.

#### A4.
*TODO: your answer here.*

#### Q5.  
Is the bufferbloat effect specific to the type of network congestion control algorithm, traffic, and/or TCP that we simulated, or is it a general phenomenon?

Are there times when increasing the router's buffer size would improve performance? If so, provide an example. If not, explain why.

#### A5.
*TODO: your answer here.*

#### Q6.  
Identify and describe a way to mitigate the bufferbloat problem without reducing the size of the buffers.

#### A6.
*TODO: your answer here.*

## Submission

**As done before, export the notebook containing all answer to either html or pdf and submit the entire folder as arxiv on the page of the course.**